In [1]:
# SETUP

import contextlib
import pathlib
import shutil

from pyiron_core.pyiron_workflow.api import util

json_file_location = util.STORED_WORKFLOW_DIR

lpg = util.LocalPostgres()
lpg.boot()

Running ('pg_ctl', '-D', '/Users/liamhuber/dev/pycor/pyiron_core/.pycor/pgdata', 'stop')
waiting for server to shut down.... done
server stopped
Running ('pg_ctl', '-D', '/Users/liamhuber/dev/pycor/pyiron_core/.pycor/pgdata', 'status')
Running ('initdb', '-D', '/Users/liamhuber/dev/pycor/pyiron_core/.pycor/pgdata')
The files belonging to this database system will be owned by user "liamhuber".
This user must also own the server process.

The database cluster will be initialized with this locale configuration:
  locale provider:   libc
  LC_COLLATE:  C
  LC_CTYPE:    UTF-8
  LC_MESSAGES: C
  LC_MONETARY: C
  LC_NUMERIC:  C
  LC_TIME:     C
The default database encoding has accordingly been set to "UTF8".
The default text search configuration will be set to "simple".

Data page checksums are disabled.

fixing permissions on existing directory /Users/liamhuber/dev/pycor/pyiron_core/.pycor/pgdata ... ok
creating subdirectories ... ok
selecting dynamic shared memory implementation ... posix


initdb: could not find suitable text search configuration for locale "UTF-8"


ok
syncing data to disk ... ok


Success. You can now start the database server using:

    pg_ctl -D /Users/liamhuber/dev/pycor/pyiron_core/.pycor/pgdata -l logfile start

Running ('pg_ctl', '-D', '/Users/liamhuber/dev/pycor/pyiron_core/.pycor/pgdata', '-o', '-p 5432', '-l', '/Users/liamhuber/dev/pycor/pyiron_core/.pycor/pgdata/logfile', 'start')
waiting for server to start.... done
server started


initdb: warning: enabling "trust" authentication for local connections
initdb: hint: You can change this by editing pg_hba.conf or using the option -A, or --auth-local and --auth-host, the next time you run initdb.


Running ('createuser', '-p', '5432', '-s', 'localuser')
Running ('createdb', '-p', '5432', '-O', 'localuser', 'localdb')
Running ('pg_ctl', '-D', '/Users/liamhuber/dev/pycor/pyiron_core/.pycor/pgdata', 'stop')
waiting for server to shut down.... done
server stopped
Running ('pg_ctl', '-D', '/Users/liamhuber/dev/pycor/pyiron_core/.pycor/pgdata', '-o', '-p 5432', '-l', '/Users/liamhuber/dev/pycor/pyiron_core/.pycor/pgdata/logfile', 'start')
waiting for server to start.... done
server started


In [ ]:
%config IPCompleter.evaluation='unsafe'

import concurrent.futures
import copy
import importlib
import json
import math
import pickle
from typing import Optional


import black
import executorlib
import numpy as np
import pandas as pd
import sqlalchemy 
import pyiron_core.pyiron_database.api as pyiron_database
from sqlalchemy import orm


from pyiron_core import nodes
from pyiron_core.pyiron_workflow.api import graph as base, simple_workflow as pwf, gui

In [ ]:
pf = gui.PyironFlow(
    ['murnaghan2', 'energy', 'murn4', 'db'], 
    hash_nodes=True, 
    db=pyiron_database.PostgreSQLInstanceDatabase(lpg.connection_string),
    workflow_path=json_file_location,
) # 'landau2',
pf.gui

In [ ]:
# MUST ENSURE `'murn4'` graph is the active one -- how to do this with code?
# We could go back and manually change the order they are loaded in,
# but I want to see what is required to actually select one
def select_graph_by_name(pf: gui.PyironFlow, graph_label: str):
    pf.tab_widget.selected_index = [w.graph.label for w in pf.wf_widgets].index(graph_label)

select_graph_by_name(pf, "murn4")

graph = pf.graph
graph = base.update_execution_graph(graph, debug=True)
nodeB = graph.nodes["BulkStaticEnergyF"].node
nodeI = graph.nodes["IterNode"].node

pyiron_database.get_hash(nodeB), id(nodeB), id(nodeI.inputs.node.value.node)

In [ ]:
id(nodeB.outputs.out.node), id(nodeB)

In [ ]:
nodeI.outputs.data.keys()

In [7]:
nodeB.inputs

,label,type,default,ready,value,node
0,name,str,NotData,True,Al,<pyiron_workflow.simple_workflow.Node object a...
1,a,float,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
2,store,bool,True,True,True,<pyiron_workflow.simple_workflow.Node object a...
3,_db,NotHinted,None,True,label ...,<pyiron_workflow.simple_workflow.Node object a...


In [8]:
nodeI.inputs.node.value

label                                                  out
type                                     builtins.NoneType
ready                                                False
value                                                 None
node     <pyiron_workflow.simple_workflow.Node object a...
dtype: object

In [ ]:
pyiron_database.get_hash(nodeI.inputs.node.value.node)

In [ ]:
 nodeI.inputs.node.value.node.inputs

In [ ]:
base._find_input_nodes(base._remove_node_inputs(graph), 1)

In [ ]:
@pwf.as_function_node("energy")
def BulkStaticEnergy(name: str, a: float = None):

    from pyiron_core import nodes
    from pyiron_core.pyiron_workflow import Workflow

    wf = Workflow('subgraph')

    wf.Bulk = nodes.atomistic.structure.build.Bulk(name=name, a=a)
    wf.M3GNet = nodes.atomistic.engine.ase.M3GNet()
    wf.Static = nodes.atomistic.calculator.ase.Static(structure=wf.Bulk, engine=wf.M3GNet)
    wf.GetEnergyLast = nodes.atomistic.calculator.output.GetEnergyLast(calculator=wf.Static)

    out = wf.GetEnergyLast.pull()

    return out

    # return wf.GetEnergyLast.outputs.energy_last

In [ ]:
@pwf.as_macro_node(["BulkStaticEnergy", "BulkStructure"])
def BulkStaticEnergy(name: str, a: float = None):

    from pyiron_core import nodes
    from pyiron_core.pyiron_workflow import Workflow

    wf = Workflow('subgraph')

    wf.Bulk = nodes.atomistic.structure.build.Bulk(name=name, a=a)
    wf.M3GNet = nodes.atomistic.engine.ase.M3GNet()
    wf.Static = nodes.atomistic.calculator.ase.Static(structure=wf.Bulk, engine=wf.M3GNet)
    wf.GetEnergyLast = nodes.atomistic.calculator.output.GetEnergyLast(calculator=wf.Static)

    return wf.GetEnergyLast.outputs.energy_last, wf.Bulk

In [ ]:
args = ["BulkStaticEnergy", "BulkStructure"]
# print("args", args, isinstance(args[0], str))
if isinstance(args[0], str):
    output_labels = list(args)
else:
    output_labels = None
output_labels

In [16]:
out = BulkStaticEnergy(name='Al').run()
out
#isinstance(out, Port)
#out.node._workflow.run()
# out._workflow.run()
# _return_as_macro_node(BulkStaticEnergy('Al'), 'Bulk', 'Energy', 'macro')

/Users/liamhuber/dev/miniforge3/envs/pycor12/lib/python3.12/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/Users/liamhuber/dev/miniforge3/envs/pycor12/lib/python3.12/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/Users/liamhuber/dev/miniforge3/envs/pycor12/lib/python3.12/site-packages/matgl/apps/pes.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().require

(-3.701530933380127,
 Al: [0. 0. 0.]
 tags: 
     indices: [0]
 pbc: [ True  True  True]
 cell: 
 Cell([[0.0, 2.025, 2.025], [2.025, 0.0, 2.025], [2.025, 2.025, 0.0]]))

In [17]:
select_graph_by_name(pf, "murnaghan2")
pf.graph.nodes["subgraph"].node.node_type

'graph'

In [18]:
base.get_inputs_of_graph(pf.graph.nodes["subgraph"].graph, exclude_unconnected_default_ports=True)

,label,value,type,default,ready
0,Bulk__name,Al,str,NotData,True
1,Bulk__a,2,float,None,True


In [19]:
print(base.get_code_from_graph(pf.graph.nodes["subgraph"].graph))


def subgraph(Bulk__name: str = "Al", Bulk__a: float = 2):

    from pyiron_workflow import Workflow
    from pyiron_nodes.atomistic.structure.build import Bulk
    from pyiron_nodes.atomistic.engine.ase import M3GNet
    from pyiron_nodes.atomistic.calculator.ase import Static
    from pyiron_nodes.atomistic.calculator.output import GetEnergyLast

    wf = Workflow('subgraph')
    wf.Bulk = Bulk(name=Bulk__name, a=Bulk__a)
    wf.M3GNet = M3GNet()
    wf.Static = Static(structure=wf.Bulk, engine=wf.M3GNet)
    wf.GetEnergyLast = GetEnergyLast(calculator=wf.Static)

    return wf.GetEnergyLast.outputs.energy_last



In [20]:
pf.db

From here on, we need the database to actually be populated with results from our workflows. Since this is not part of our static data, obtain it by simply re-running each of the existing workflows.

In [21]:
def pull_all_terminal_nodes(graph: base.Graph, _verbose: bool = False):
    graph_copy = base.get_updated_graph(graph)
    for label, graph_node in graph_copy.nodes.items():
        is_virtual = (
            graph_node.node_type != "node" # Is this true? Is a "graph" node_type always virtual?
            and graph_node.node is not None  # These are virtual IO nodes
        )
        is_terminal = label not in graph.edges.df["source"].values
        
        if is_terminal and not is_virtual:
            if _verbose:
                print("Pulling", label)
            base.pull_node(graph_copy, label, db=pf.db)

for wfw in pf.wf_widgets:
    if wfw.graph.label == "db":
        # The GetGraph node of this workflow fails for the stored "node_id" of 20
        # It also fails if I go back and set this to 0
        # I don't know what should be in the database at this point, but it isn't
        continue
    print("Ensuring execution for", wfw.graph.label)
    pull_all_terminal_nodes(wfw.graph)

/Users/liamhuber/dev/miniforge3/envs/pycor12/lib/python3.12/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/Users/liamhuber/dev/miniforge3/envs/pycor12/lib/python3.12/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/Users/liamhuber/dev/miniforge3/envs/pycor12/lib/python3.12/site-packages/matgl/apps/pes.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().require

Ensuring execution for murnaghan2
Running node Linspace
Running node CreateDB
Running node IterNode
Restoring node outputs  9bd7911eb91f5326720e2753be05f5fde5a2edef5bc7c0d66e12817ffb145501 IterNode False
No stored data found for node:  IterNode
copy node (port):  subgraph fb6471b1fc8bd5750d43fac2f3f132e61276aec65b34f665550cafe024c01e2a


DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
/Users/liamhuber/dev/miniforge3/envs/pycor12/lib/python3.12/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/Users/liamhuber/dev/miniforge3/envs/pycor12/lib/python3.12/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", to

serialization needed
Running node CreateDB
Running node ShowTable
Ensuring execution for energy
Running node Bulk
Running node M3GNet
Running node CreateDB
Running node Static
Running node GetEnergyLast
Restoring node outputs  9b661e5ccaf7bf011f06f2b3254809d47baeb9632fa51454bfaf62f7c00ad89e GetEnergyLast False
No stored data found for node:  GetEnergyLast
serialization not needed
Running node CreateDB
Running node ShowTable
Ensuring execution for murn4
Running node Linspace
Running node CreateDB
Running node IterNode
Restoring node outputs  f5acb73836985369d523b5587504f9860c2fe7ce84c027e6f2d2ffb6e655a28d IterNode False
No stored data found for node:  IterNode
copy node (port):  BulkStaticEnergyF 53ac213be901c9c01914b0184ad75b68bed31310fdbf559557cc1a9fce9e025d
Restoring node outputs  2ce3c3a3d5a648f7bd704d950f60cc384fe8caa26f9624777782003cd273dffc BulkStaticEnergyF False
No stored data found for node:  BulkStaticEnergyF
serialization not needed
Restoring node outputs  276310e6b46125c446

DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
/Users/liamhuber/dev/miniforge3/envs/pycor12/lib/python3.12/site-packages/dgl/core.py:82: DGLWarning: The input graph for the user-defined edge function does not contain valid edges
  dgl_warning(
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!


serialization not needed
Restoring node outputs  b60f6d99ddffb5207c332b1e74690acb40681935b1895136516bbe8e78f16213 BulkStaticEnergyF False
No stored data found for node:  BulkStaticEnergyF
serialization not needed
Restoring node outputs  739d468e8a6545995369fb7c9c9e13d2fd367c0a3505b3fb40dd3f583db369b6 BulkStaticEnergyF False
No stored data found for node:  BulkStaticEnergyF
serialization not needed
Restoring node outputs  5b7e6a6570012f882b7b6af8c682d4e820c0bcd1738c6d231680492495c60ed9 BulkStaticEnergyF False
No stored data found for node:  BulkStaticEnergyF
serialization not needed
Restoring node outputs  44b1dc2862608dd3150b77d8efcbba02be1ec99a6f328d7d02928ccd6873ba02 BulkStaticEnergyF False
No stored data found for node:  BulkStaticEnergyF
serialization not needed
serialization not needed
serialization needed
Running node CreateDB
Running node ShowTable


In [22]:
select_graph_by_name(pf, "murn4")
pf.graph.nodes["ShowTable"].node.outputs.df.value

,hash,qualname,module,version,connected_inputs,inputs,outputs,output_path,master_hash,start_time,cpu_time,executor,user
0,39669a289cc8b85c2e0774812b662253c661d306ef02d9...,subgraph,pyiron_workflow.graph.base,not_defined,[],"{'Bulk__a': 2.2, 'Bulk__name': 'Al'}",[GetEnergyLast__energy_last],None,fb6471b1fc8bd5750d43fac2f3f132e61276aec65b34f6...,2025-08-12 07:47:44.230099,0.049834,None,liamhuber
1,11e5f779d5a00821a3a28813c8cdb0c83a38c1fedce8e6...,Linspace,pyiron_nodes.math,not_defined,[],"{'x_max': 2.2, 'x_min': 1.8, 'endpoint': True,...",[linspace],None,None,2025-08-12 07:47:43.659189,0.000048,None,liamhuber
2,64667395d1b66a8a0ef131b4bade730b217b32465b813b...,CreateDB,pyiron_nodes.databases.node_hash_db,not_defined,[],"{'host': 'localhost', 'port': 5432, 'user': 'l...",[db],None,None,2025-08-12 07:47:43.659255,0.058096,None,liamhuber
3,10cbd2c7503397244d3b8ba7eda766aca96cb845aee39e...,IterNode,pyiron_nodes.executors,not_defined,"[node, kwarg_list, _db]",{'_db': '64667395d1b66a8a0ef131b4bade730b217b3...,[df],/Users/liamhuber/.storage/10cbd2c7503397244d3b...,None,2025-08-12 07:47:43.753743,0.526325,None,liamhuber
4,7b4123d2f2ee2e9877493b62f6becb05f0483761d43031...,Bulk,pyiron_nodes.atomistic.structure.build,not_defined,[],"{'a': None, 'c': None, 'u': None, 'name': 'Al'...",[structure],None,None,2025-08-12 07:47:44.303577,0.000211,None,liamhuber
5,635cc10c63f0d96564a0eeb790d4adf102485cc41e475f...,M3GNet,pyiron_nodes.atomistic.engine.ase,not_defined,[],{'model': 'M3GNet-MP-2021.2.8-PES'},[out],None,None,2025-08-12 07:47:44.303800,0.016429,None,liamhuber
6,29624f6695f9b692420ba8f88e6a7965fff1347d9148b9...,Static,pyiron_nodes.atomistic.calculator.ase,not_defined,"[structure, engine]",{'engine': '635cc10c63f0d96564a0eeb790d4adf102...,[out],None,None,2025-08-12 07:47:44.325495,0.008331,None,liamhuber
7,9b661e5ccaf7bf011f06f2b3254809d47baeb9632fa514...,GetEnergyLast,pyiron_nodes.atomistic.calculator.output,not_defined,"[calculator, _db]",{'_db': '64667395d1b66a8a0ef131b4bade730b217b3...,[energy_last],/Users/liamhuber/.storage/9b661e5ccaf7bf011f06...,None,2025-08-12 07:47:44.341191,0.003863,None,liamhuber
8,2ce3c3a3d5a648f7bd704d950f60cc384fe8caa26f9624...,BulkStaticEnergyF,pyiron_nodes.local_workflows.test,not_defined,[_db],"{'a': 3.0, '_db': '64667395d1b66a8a0ef131b4bad...",[out],/Users/liamhuber/.storage/2ce3c3a3d5a648f7bd70...,53ac213be901c9c01914b0184ad75b68bed31310fdbf55...,2025-08-12 07:47:44.377746,0.035790,None,liamhuber
9,276310e6b46125c446f61274ed951cd586592522fd3d9b...,BulkStaticEnergyF,pyiron_nodes.local_workflows.test,not_defined,[_db],"{'a': 3.6666666666666665, '_db': '64667395d1b6...",[out],/Users/liamhuber/.storage/276310e6b46125c446f6...,53ac213be901c9c01914b0184ad75b68bed31310fdbf55...,2025-08-12 07:47:44.423925,0.030003,None,liamhuber


In [23]:
select_graph_by_name(pf, "murn4")
pf.graph.nodes["ShowTable"].node.outputs.df.value.iloc[7].hash

'9b661e5ccaf7bf011f06f2b3254809d47baeb9632fa51454bfaf62f7c00ad89e'

In [24]:
db = pf.db
# db.drop()  # But if we do this, we lose all our data?!
# db.init()



In [ ]:
Session = orm.sessionmaker(bind=db.engine)
session = Session()

df = pd.read_sql(session.query(db.table).statement, session.bind)

session.close()

_, graph = pyiron_database.restore_node_from_database(db=db, node_hash=df.hash.iloc[6]) #[19])
#_, graph = restore_node_from_database(db=db, node_hash='cf0bda97885d0ddf6b8a4cd1a92d38806147e0721a9f646cfb3a37487dd6a7b2')
gui.GuiGraph(graph)

In [ ]:
df

In [27]:
# df.query("hash == '9c051a26834ef83bbbe6c11c9fc0fc8c24cc06229432d31021fd00faa4c738ef'").iloc[0]
# Couldn't find that hash
some_valid_hash = df["hash"].iloc[0]
df.query(f"hash == '{some_valid_hash}'").iloc[0]

hash                39669a289cc8b85c2e0774812b662253c661d306ef02d9...
qualname                                                     subgraph
module                                     pyiron_workflow.graph.base
version                                                   not_defined
connected_inputs                                                   []
inputs                           {'Bulk__a': 2.2, 'Bulk__name': 'Al'}
outputs                                  [GetEnergyLast__energy_last]
output_path                                                      None
master_hash         fb6471b1fc8bd5750d43fac2f3f132e61276aec65b34f6...
start_time                                 2025-08-12 07:47:44.230099
cpu_time                                                     0.049834
executor                                                         None
user                                                        liamhuber
Name: 0, dtype: object

In [28]:
# df[df.hash==inputs["node"].split("@")[0]].iloc[0]
# NameError: name 'inputs' is not defined

In [29]:
# button
# NameError: name 'button' is not defined

In [ ]:
nodes.atomistic.structure.build.Bulk('Al')._func.__qualname__

In [ ]:
select_graph_by_name(pf, "murn4")
df = pf.graph.nodes["IterNode"].node.outputs.df.value
state = df.__getstate__()
print(state)

df_new = pd.DataFrame() # .__setstate__(state)
df_new._mgr = state["_mgr"]
df_new._metadata = state["_metadata"]
df_new2 = pd.DataFrame()
df_new2.__setstate__(state)
df_new2

In [32]:
node = pf.graph.nodes["IterNode"].node

In [33]:
user = 'localuser'
password = 'none'
host = 'localhost'
port = '5432'
database = 'localdb'

connection_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'
engine = sqlalchemy.create_engine(connection_str)
engine.connect()

inspector = sqlalchemy.inspect(engine)
table_names = inspector.get_table_names()
table_names

['nodes']

In [ ]:
node = pf.graph.nodes["IterNode"].node

db = pyiron_database.PostgreSQLInstanceDatabase(connection_str)
# db.drop()
# db.init()
# hash = idb.store_node_in_database(db, node, store_outputs=True, store_input_nodes_recursively=True)
# Gets a pickle complaint -- just skipping running it:
# File ~/dev/pycor/pyiron_core/pyiron_database/obj_reconstruction/util.py:114, in pickle_dump(obj)
#     113 def pickle_dump(obj):
# --> 114     return codecs.encode(pickle.dumps(obj), "base64").decode()
# AttributeError: Can't get local object 'create_engine.<locals>.connect'

# idb.store_node_outputs(pf.graph.nodes["IterNode"].node)

In [ ]:
type(db)

In [36]:
Session = orm.sessionmaker(bind=db.engine)
session = Session()

df = pd.read_sql(session.query(db.table).statement, session.bind)

df

,hash,qualname,module,version,connected_inputs,inputs,outputs,output_path,master_hash,start_time,cpu_time,executor,user
0,39669a289cc8b85c2e0774812b662253c661d306ef02d9...,subgraph,pyiron_workflow.graph.base,not_defined,[],"{'Bulk__a': 2.2, 'Bulk__name': 'Al'}",[GetEnergyLast__energy_last],None,fb6471b1fc8bd5750d43fac2f3f132e61276aec65b34f6...,2025-08-12 07:47:44.230099,0.049834,None,liamhuber
1,11e5f779d5a00821a3a28813c8cdb0c83a38c1fedce8e6...,Linspace,pyiron_nodes.math,not_defined,[],"{'x_max': 2.2, 'x_min': 1.8, 'endpoint': True,...",[linspace],None,None,2025-08-12 07:47:43.659189,0.000048,None,liamhuber
2,64667395d1b66a8a0ef131b4bade730b217b32465b813b...,CreateDB,pyiron_nodes.databases.node_hash_db,not_defined,[],"{'host': 'localhost', 'port': 5432, 'user': 'l...",[db],None,None,2025-08-12 07:47:43.659255,0.058096,None,liamhuber
3,10cbd2c7503397244d3b8ba7eda766aca96cb845aee39e...,IterNode,pyiron_nodes.executors,not_defined,"[node, kwarg_list, _db]",{'_db': '64667395d1b66a8a0ef131b4bade730b217b3...,[df],/Users/liamhuber/.storage/10cbd2c7503397244d3b...,None,2025-08-12 07:47:43.753743,0.526325,None,liamhuber
4,7b4123d2f2ee2e9877493b62f6becb05f0483761d43031...,Bulk,pyiron_nodes.atomistic.structure.build,not_defined,[],"{'a': None, 'c': None, 'u': None, 'name': 'Al'...",[structure],None,None,2025-08-12 07:47:44.303577,0.000211,None,liamhuber
5,635cc10c63f0d96564a0eeb790d4adf102485cc41e475f...,M3GNet,pyiron_nodes.atomistic.engine.ase,not_defined,[],{'model': 'M3GNet-MP-2021.2.8-PES'},[out],None,None,2025-08-12 07:47:44.303800,0.016429,None,liamhuber
6,29624f6695f9b692420ba8f88e6a7965fff1347d9148b9...,Static,pyiron_nodes.atomistic.calculator.ase,not_defined,"[structure, engine]",{'engine': '635cc10c63f0d96564a0eeb790d4adf102...,[out],None,None,2025-08-12 07:47:44.325495,0.008331,None,liamhuber
7,9b661e5ccaf7bf011f06f2b3254809d47baeb9632fa514...,GetEnergyLast,pyiron_nodes.atomistic.calculator.output,not_defined,"[calculator, _db]",{'_db': '64667395d1b66a8a0ef131b4bade730b217b3...,[energy_last],/Users/liamhuber/.storage/9b661e5ccaf7bf011f06...,None,2025-08-12 07:47:44.341191,0.003863,None,liamhuber
8,2ce3c3a3d5a648f7bd704d950f60cc384fe8caa26f9624...,BulkStaticEnergyF,pyiron_nodes.local_workflows.test,not_defined,[_db],"{'a': 3.0, '_db': '64667395d1b66a8a0ef131b4bad...",[out],/Users/liamhuber/.storage/2ce3c3a3d5a648f7bd70...,53ac213be901c9c01914b0184ad75b68bed31310fdbf55...,2025-08-12 07:47:44.377746,0.035790,None,liamhuber
9,276310e6b46125c446f61274ed951cd586592522fd3d9b...,BulkStaticEnergyF,pyiron_nodes.local_workflows.test,not_defined,[_db],"{'a': 3.6666666666666665, '_db': '64667395d1b6...",[out],/Users/liamhuber/.storage/276310e6b46125c446f6...,53ac213be901c9c01914b0184ad75b68bed31310fdbf55...,2025-08-12 07:47:44.423925,0.030003,None,liamhuber


In [37]:
# db.drop()
# db.init()

In [38]:
pf = gui.PyironFlow(['assyst', 'linearfit2', 'graph', 'executor', 'executor2', 'murnaghan', 'murnaghan2'], workflow_path=json_file_location) # 'landau2',
pf.gui

In [39]:
# pf = gui.PyironFlow(['assyst', 'calphy2', 'linearfit2', 'graph', 'executor', 'executor2', 'murnaghan', 'murnaghan2']) # 'landau2',
# pf.gui

# https://github.com/JNmpi/pyiron_core/issues/52
# The calphy2 graph will not load:
# File ~/dev/pycor/pyiron_core/pyiron_core.pyiron_workflow/graph/base.py:58, in _setstate__graph_node(self, state)
#      57 def _setstate__graph_node(self, state):
# ---> 58     for k, v in state.items():
#      59         if k == "node":
#      60             # print("setting node: ", k, v)
#      61             # check if virtual node (import not possible) # TODO: make more robust test
#      62             if v["function"].startswith("pyiron_core.pyiron_workflow.graph.base"):
#      63                 # graph = Graph().__setstate__(state["graph"])

# AttributeError: 'NoneType' object has no attribute 'items'

In [40]:
select_graph_by_name(pf, "murnaghan2")
# could also be "executor2" -- both have "subgraph" and "Linspace" nodes
# ("calphy2" didn't load, but I can peek at the json file to rule it out, as it has neither)
label ="subgraph"
label = "Linspace"
pf.graph.nodes[label].node.copy()._graph_node == pf.graph.nodes[label]

True

In [41]:
new_node = pickle.loads(pickle.dumps(pf.graph.nodes["subgraph"]))
new_node.node

In [42]:
base.GraphNode().__setstate__(pf.graph.nodes["subgraph"].__getstate__()).node.run()

/Users/liamhuber/dev/miniforge3/envs/pycor12/lib/python3.12/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/Users/liamhuber/dev/miniforge3/envs/pycor12/lib/python3.12/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/Users/liamhuber/dev/miniforge3/envs/pycor12/lib/python3.12/site-packages/matgl/apps/pes.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().require

53.408348083496094

In [ ]:
wf = pwf.Workflow("test")
wf.bulk = nodes.atomistic.structure.build.Bulk("Al", cubic=True)
wf.volume = nodes.atomistic.structure.calc.Volume(wf.bulk)
# wf.subgraph = Group([wf.bulk, wf.volume])

wf.range = nodes.math.Linspace(1.8, 2.2, 7)
# wf.executor = nodes.executor.
# wf.iter = nodes.executors.IterNode(node=wf.volume, kwarg_name="Bulk__name", kwarg_list=wf.range)

graph = base.get_graph_from_wf(wf, [], [])
graph_sub = base.create_group(graph, [0, 2])

base.get_updated_graph(graph_sub)
#graph_sub

In [ ]:
# graph.nodes["test"] = base.GraphNode()

In [45]:
wf_file = json_file_location / "murnaghan2.json"
with open(wf_file, "r") as f:
    graph_dict = (json.load(f))

# base.Nodes().__setstate__(graph_dict["nodes"])
sub_nodes= graph_dict["nodes"]["subgraph"]["graph"]["nodes"]
#base.Nodes().__setstate__(sub_nodes)
sub_nodes

{'Bulk': {'id': 'Bulk',
  'import_path': 'pyiron_nodes.atomistic.structure.build.Bulk',
  'label': 'Bulk',
  'parent_id': None,
  'level': 0,
  'graph': None,
  'node_type': 'node',
  'widget_type': 'customNode',
  'expanded': False,
  'node': {'label': 'Bulk',
   'function': 'pyiron_nodes.atomistic.structure.build.Bulk',
   'inputs': {'name': 'Al', 'a': 2}}},
 'M3GNet': {'id': 'M3GNet',
  'import_path': 'pyiron_nodes.atomistic.engine.ase.M3GNet',
  'label': 'M3GNet',
  'parent_id': None,
  'level': 0,
  'graph': None,
  'node_type': 'node',
  'widget_type': 'customNode',
  'expanded': False,
  'node': {'label': 'M3GNet',
   'function': 'pyiron_nodes.atomistic.engine.ase.M3GNet',
   'inputs': {}}},
 'Static': {'id': 'Static',
  'import_path': 'pyiron_nodes.atomistic.calculator.ase.Static',
  'label': 'Static',
  'parent_id': None,
  'level': 0,
  'graph': None,
  'node_type': 'node',
  'widget_type': 'customNode',
  'expanded': False,
  'node': {'label': 'Static',
   'function': 'pyiro

In [46]:
wf_file = json_file_location / "murnaghan2.json"
with open(wf_file, "r") as f:
    graph_dict = (json.load(f))


    
 
# loaded_graph = base.Graph().__setstate__(graph_dict["nodes"]["subgraph"]["graph"])
loaded_graph = base.Graph().__setstate__(graph_dict)
#print(get_code_from_graph(loaded_graph.nodes["subgraph"].graph))


# print(get_code_from_graph(loaded_graph.nodes["subgraph"]))
#loaded_graph.nodes["subgraph"].node

#base.graph_to_node(base.Graph().__setstate__(graph_dict["nodes"]["subgraph"]["graph"]))
subgraph = base.Graph().__setstate__(graph_dict["nodes"]["subgraph"]["graph"])

#get_code_from_graph(subgraph)
subgraph.nodes._obj_type is None
#subgraph

False

In [47]:
graph_dict["nodes"]["subgraph"]["graph"]["nodes"]

{'Bulk': {'id': 'Bulk',
  'import_path': 'pyiron_nodes.atomistic.structure.build.Bulk',
  'label': 'Bulk',
  'parent_id': None,
  'level': 0,
  'graph': None,
  'node_type': 'node',
  'widget_type': 'customNode',
  'expanded': False,
  'node': {'label': 'Bulk',
   'function': 'pyiron_nodes.atomistic.structure.build.Bulk',
   'inputs': {'name': 'Al', 'a': 2}}},
 'M3GNet': {'id': 'M3GNet',
  'import_path': 'pyiron_nodes.atomistic.engine.ase.M3GNet',
  'label': 'M3GNet',
  'parent_id': None,
  'level': 0,
  'graph': None,
  'node_type': 'node',
  'widget_type': 'customNode',
  'expanded': False,
  'node': {'label': 'M3GNet',
   'function': 'pyiron_nodes.atomistic.engine.ase.M3GNet',
   'inputs': {}}},
 'Static': {'id': 'Static',
  'import_path': 'pyiron_nodes.atomistic.calculator.ase.Static',
  'label': 'Static',
  'parent_id': None,
  'level': 0,
  'graph': None,
  'node_type': 'node',
  'widget_type': 'customNode',
  'expanded': False,
  'node': {'label': 'Static',
   'function': 'pyiro

In [48]:
loaded_graph.nodes._obj_type

pyiron_workflow.graph.base.GraphNode

The decorator functions of Nodes are a constant source of errors. Can we make them more elegant and robust by going from decorator arguments to class inheritance?? Do I really need something like ob_type? I know the type I want to get from the name, i.e., nodes should have always GraphNode elements.

In [49]:
subgraph = base._load_graph("murnaghan2", workflow_dir=json_file_location).nodes["subgraph"]
print('xxx: ', base.graph_to_code(subgraph.graph))

gui.GuiGraph(subgraph.graph)

xxx:  
def subgraph(Bulk__name: str = "Al", Bulk__a: int = 2):

    from pyiron_workflow import Workflow
    from pyiron_nodes.atomistic.structure.build import Bulk
    from pyiron_nodes.atomistic.engine.ase import M3GNet
    from pyiron_nodes.atomistic.calculator.ase import Static
    from pyiron_nodes.atomistic.calculator.output import GetEnergyLast

    wf = Workflow('subgraph')
    wf.Bulk = Bulk(name=Bulk__name, a=Bulk__a)
    wf.M3GNet = M3GNet()
    wf.Static = Static(structure=wf.Bulk, engine=wf.M3GNet)
    wf.GetEnergyLast = GetEnergyLast(calculator=wf.Static)

    return wf.GetEnergyLast.outputs.energy_last



ReactFlowWidget(layout=Layout(height='600px', width='800px'))

In [50]:
wf_file = json_file_location / "murnaghan2.json"
with open(wf_file, "r") as f:
    graph_dict = (json.load(f))

loaded_graph = base.Graph().__setstate__(graph_dict)

subgraph2 = loaded_graph.nodes["subgraph"].graph
print('xxx: ', base.graph_to_code(subgraph2))
gui.GuiGraph(subgraph2)

xxx:  
def subgraph(Bulk__name: str = "Al", Bulk__a: int = 2):

    from pyiron_workflow import Workflow
    from pyiron_nodes.atomistic.structure.build import Bulk
    from pyiron_nodes.atomistic.engine.ase import M3GNet
    from pyiron_nodes.atomistic.calculator.ase import Static
    from pyiron_nodes.atomistic.calculator.output import GetEnergyLast

    wf = Workflow('subgraph')
    wf.Bulk = Bulk(name=Bulk__name, a=Bulk__a)
    wf.M3GNet = M3GNet()
    wf.Static = Static(structure=wf.Bulk, engine=wf.M3GNet)
    wf.GetEnergyLast = GetEnergyLast(calculator=wf.Static)

    return wf.GetEnergyLast.outputs.energy_last



ReactFlowWidget(layout=Layout(height='600px', width='800px'))

In [51]:
print(base.get_code_from_graph(loaded_graph.nodes["subgraph"].graph))


def subgraph(Bulk__name: str = "Al", Bulk__a: int = 2):

    from pyiron_workflow import Workflow
    from pyiron_nodes.atomistic.structure.build import Bulk
    from pyiron_nodes.atomistic.engine.ase import M3GNet
    from pyiron_nodes.atomistic.calculator.ase import Static
    from pyiron_nodes.atomistic.calculator.output import GetEnergyLast

    wf = Workflow('subgraph')
    wf.Bulk = Bulk(name=Bulk__name, a=Bulk__a)
    wf.M3GNet = M3GNet()
    wf.Static = Static(structure=wf.Bulk, engine=wf.M3GNet)
    wf.GetEnergyLast = GetEnergyLast(calculator=wf.Static)

    return wf.GetEnergyLast.outputs.energy_last



In [52]:
subgraph = base._load_graph("murnaghan2", workflow_dir=json_file_location).nodes["subgraph"]
print('xxx: ', base.graph_to_code(subgraph.graph))

xxx:  
def subgraph(Bulk__name: str = "Al", Bulk__a: int = 2):

    from pyiron_workflow import Workflow
    from pyiron_nodes.atomistic.structure.build import Bulk
    from pyiron_nodes.atomistic.engine.ase import M3GNet
    from pyiron_nodes.atomistic.calculator.ase import Static
    from pyiron_nodes.atomistic.calculator.output import GetEnergyLast

    wf = Workflow('subgraph')
    wf.Bulk = Bulk(name=Bulk__name, a=Bulk__a)
    wf.M3GNet = M3GNet()
    wf.Static = Static(structure=wf.Bulk, engine=wf.M3GNet)
    wf.GetEnergyLast = GetEnergyLast(calculator=wf.Static)

    return wf.GetEnergyLast.outputs.energy_last



In [53]:
base._build_function_parameters(subgraph.graph, use_node_default=False)

'Bulk__name: str = "Al", Bulk__a: int = 2'

In [54]:
print('xxx: ', base.get_code_from_graph(subgraph.graph, use_node_default=False))

xxx:  
def subgraph(Bulk__name: str = "Al", Bulk__a: int = 2):

    from pyiron_workflow import Workflow
    from pyiron_nodes.atomistic.structure.build import Bulk
    from pyiron_nodes.atomistic.engine.ase import M3GNet
    from pyiron_nodes.atomistic.calculator.ase import Static
    from pyiron_nodes.atomistic.calculator.output import GetEnergyLast

    wf = Workflow('subgraph')
    wf.Bulk = Bulk(name=Bulk__name, a=Bulk__a)
    wf.M3GNet = M3GNet()
    wf.Static = Static(structure=wf.Bulk, engine=wf.M3GNet)
    wf.GetEnergyLast = GetEnergyLast(calculator=wf.Static)

    return wf.GetEnergyLast.outputs.energy_last



In [55]:
graph = base._load_graph("murnaghan2", workflow_dir=json_file_location)
print(base.get_code_from_graph(graph.nodes["subgraph"].graph))


def subgraph(Bulk__name: str = "Al", Bulk__a: int = 2):

    from pyiron_workflow import Workflow
    from pyiron_nodes.atomistic.structure.build import Bulk
    from pyiron_nodes.atomistic.engine.ase import M3GNet
    from pyiron_nodes.atomistic.calculator.ase import Static
    from pyiron_nodes.atomistic.calculator.output import GetEnergyLast

    wf = Workflow('subgraph')
    wf.Bulk = Bulk(name=Bulk__name, a=Bulk__a)
    wf.M3GNet = M3GNet()
    wf.Static = Static(structure=wf.Bulk, engine=wf.M3GNet)
    wf.GetEnergyLast = GetEnergyLast(calculator=wf.Static)

    return wf.GetEnergyLast.outputs.energy_last



In [56]:
# base.Graph(id=None, 
#       label='subgraph', 
#       root_node=None, 
#       nodes=NestedDict({'Bulk': GraphNode(id='Bulk', import_path='pyiron_nodes.atomistic.structure.build.Bulk', label='Bulk', 
#                                           parent_id=None, level=0, node=<pyiron_core.pyiron_workflow.simple_workflow.Node object at 0x17372c3e0>,
#                                           graph=None, node_type='node', widget_type='customNode', expanded=False), 
#                         'M3GNet': GraphNode(id='M3GNet', import_path='pyiron_nodes.atomistic.engine.ase.M3GNet', label='M3GNet', 
#                                             parent_id=None, level=0, node=<pyiron_core.pyiron_workflow.simple_workflow.Node object at 0x17372c410>,
#                                             graph=None, node_type='node', widget_type='customNode', expanded=False), 
#                         'Static': GraphNode(id='Static', import_path='pyiron_nodes.atomistic.calculator.ase.Static', label='Static', 
#                                             parent_id=None, level=0, node=<pyiron_core.pyiron_workflow.simple_workflow.Node object at 0x17372c0e0>,
#                                             graph=None, node_type='node', widget_type='customNode', expanded=False), 
#                         'GetEnergyLast': GraphNode(id='GetEnergyLast', import_path='pyiron_nodes.atomistic.calculator.output.GetEnergyLast', label='GetEnergyLast', 
#                                                    parent_id=None, level=0, node=<pyiron_core.pyiron_workflow.simple_workflow.Node object at 0x17372c3b0>,
#                                                    graph=None, node_type='node', widget_type='customNode', expanded=False)}), 
#       edges=[GraphEdge(source='Static', target='GetEnergyLast', sourceHandle='out', targetHandle='calculator'), 
#              GraphEdge(source='Bulk', target='Static', sourceHandle='structure', targetHandle='structure'), 
#              GraphEdge(source='M3GNet', target='Static', sourceHandle='out', targetHandle='engine')], graph={})

# parent_id=None, level=0, node=<pyiron_core.pyiron_workflow.simple_workflow.Node object at 0x17372c3e0>,
#                                   ^
# SyntaxError: invalid syntax

# This appears to be a paste of a copied print and not actual code

In [57]:
subgraph.graph

Graph(id=None, label='subgraph', root_node=None, nodes=Nodes({'Bulk': GraphNode(id='Bulk', import_path='pyiron_nodes.atomistic.structure.build.Bulk', label='Bulk', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33af91b80>, graph=None, node_type='node', widget_type='customNode', expanded=False), 'M3GNet': GraphNode(id='M3GNet', import_path='pyiron_nodes.atomistic.engine.ase.M3GNet', label='M3GNet', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33af50620>, graph=None, node_type='node', widget_type='customNode', expanded=False), 'Static': GraphNode(id='Static', import_path='pyiron_nodes.atomistic.calculator.ase.Static', label='Static', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33af91be0>, graph=None, node_type='node', widget_type='customNode', expanded=False), 'GetEnergyLast': GraphNode(id='GetEnergyLast', import_path='pyiron_nodes.atomistic.calculator.output.GetEnergyLast', label='Get

In [58]:
subgraph.node.inputs

,label,value,type,default,ready,node
0,Bulk__name,Al,str,NotData,True,<pyiron_workflow.simple_workflow.Node object a...
1,Bulk__a,2,int,None,True,<pyiron_workflow.simple_workflow.Node object a...


In [59]:
graph2 = base._load_graph("executor3", workflow_dir=json_file_location)
for node in graph2.nodes.values():
    print(node)

GraphNode(id='Bulk', import_path='pyiron_nodes.atomistic.structure.build.Bulk', label='Bulk', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33afeca70>, graph=None, node_type='node', widget_type='customNode', expanded=False)
GraphNode(id='Volume', import_path='pyiron_nodes.atomistic.structure.calc.Volume', label='Volume', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33afec410>, graph=None, node_type='node', widget_type='customNode', expanded=False)
GraphNode(id='ThreadPoolExecutor', import_path='pyiron_nodes.executors.ThreadPoolExecutor', label='ThreadPoolExecutor', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33afec500>, graph=None, node_type='node', widget_type='customNode', expanded=False)
GraphNode(id='IterNode', import_path='pyiron_nodes.executors.IterNode', label='IterNode', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33afee150>, graph=None, n

In [ ]:
%%time
Al = nodes.atomistic.structure.build.CubicBulkCell(element="Al")

dumps = pickle.dumps((Al, base.run_node))

Al.__getstate__()
Al_p, run = pickle.loads(dumps)
Al_p.inputs
# Al
pwf.Node().__setstate__(state=Al.__getstate__())

In [ ]:
graph = base._load_graph("executor3", workflow_dir=json_file_location)

# Originally this resulted in KeyError: 'subgraph'
# Indeed, the executor3.json file has no subgraph.
# Lets just stick _all_ the nodes in a subgraph and proceed
# Nevermind, nope, because we're still not allowed multiple unconnected outputs in a subgraph
# Just make a bespoke selection for now......
graph = base.create_group(graph, [0, 1, 2, 3, 5], label="subgraph")

subgraph = graph.nodes["subgraph"].graph
#base.graph_to_node(subgraph)
for node in subgraph.nodes.values():
    print(type(node))
    
    # g = base.GraphNode().__setstate__(node)
    # TypeError: 'Node' object is not subscriptable
    
    g = base.GraphNode().__setstate__(node.__getstate__())
    print('NODE: ', g.label)

In [62]:
# graph = base._load_graph("executor3")
# subgraph = graph.nodes["subgraph"].graph
# #base.graph_to_node(subgraph)
# for node in subgraph.nodes.values():
#     g = base.GraphNode().__setstate__(node)
#     print('NODE: ', g.label)

# Duplicate cell, just comment it out

In [ ]:
module = importlib.import_module('pyiron_core.pyiron_nodes.atomistic.structure.build')
module.Bulk()

In [ ]:
pwf.get_node_from_path('pyiron_core.pyiron_nodes.atomistic.structure.build.Bulk')()

In [ ]:
state = pickle.dumps(Al)
state

In [66]:
copy.copy(Al_p).inputs

,label,type,default,ready,value,node
0,element,str,NotData,True,Al,<pyiron_workflow.simple_workflow.Node object a...
1,cell_size,int,1,True,1,<pyiron_workflow.simple_workflow.Node object a...
2,vacancy_index,int,None,True,None,<pyiron_workflow.simple_workflow.Node object a...


In [67]:
graph = base._load_graph("bulk", workflow_dir=json_file_location)
# base._nodes_to_gui(graph)
#graph.nodes #["Bulk"]
graph

Graph(id=None, label='bulk', root_node=None, nodes=Nodes({'Bulk': GraphNode(id='Bulk', import_path='pyiron_nodes.atomistic.structure.build.Bulk', label='Bulk', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33afefe00>, graph=None, node_type='node', widget_type='customNode', expanded=False)}), edges=[], graph={})

In [68]:
node = pwf.Node().__setstate__(state=Al.__getstate__())
node.inputs

,label,type,default,ready,value,node
0,element,str,NotData,True,Al,<pyiron_workflow.simple_workflow.Node object a...
1,cell_size,int,1,True,1,<pyiron_workflow.simple_workflow.Node object a...
2,vacancy_index,int,None,True,None,<pyiron_workflow.simple_workflow.Node object a...


In [69]:
pf = gui.PyironFlow(
    [
        'assyst', 
        # 'calphy2',   # AttributeError: 'NoneType' object has no attribute 'items'
        # https://github.com/JNmpi/pyiron_core/issues/52
        'linearfit2', 
        'graph', 
        'executor', 
        'executor3',
    ],
    workflow_path=json_file_location,
) # 'landau2',
pf.gui

In [70]:
select_graph_by_name(pf, "linearfit2")
# Selecting the one we want doesn't work when it's already the one selected
# Select something else first, then go back
select_graph_by_name(pf, "assyst")
# pf.graph only gets populated after switching tabs in the GUI
# here just take the first one in the list and see if it works

graph = pf.graph #.__getstate__()


# base._get_subgraph(graph, [0,1]).nodes.__getstate__()
graph.nodes.iloc(1).__getstate__()

{'id': 'SpaceGroupInput',
 'import_path': 'pyiron_nodes.atomistic.assyst.structures.SpaceGroupInput',
 'label': 'SpaceGroupInput',
 'parent_id': None,
 'level': 0,
 'graph': None,
 'node_type': 'node',
 'widget_type': 'customNode',
 'expanded': False,
 'node': {'label': 'SpaceGroupInput',
  'function': 'pyiron_nodes.atomistic.assyst.structures.SpaceGroupInput',
  'inputs': {'max_structures': 100}}}

In [71]:
graph

Graph(id=None, label='assyst', root_node=None, nodes=Nodes({'ElementInput': GraphNode(id='ElementInput', import_path='pyiron_nodes.atomistic.assyst.structures.ElementInput', label='ElementInput', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33b810770>, graph=None, node_type='node', widget_type='customNode', expanded=False), 'SpaceGroupInput': GraphNode(id='SpaceGroupInput', import_path='pyiron_nodes.atomistic.assyst.structures.SpaceGroupInput', label='SpaceGroupInput', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33af50980>, graph=None, node_type='node', widget_type='customNode', expanded=False), 'SpaceGroupSampling': GraphNode(id='SpaceGroupSampling', import_path='pyiron_nodes.atomistic.assyst.structures.SpaceGroupSampling', label='SpaceGroupSampling', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33af90fe0>, graph=None, node_type='node', widget_type='customNode', expanded=False), '

In [72]:
# pf.graph.nodes["subgraph"].__getstate__()
# None of the saved graphs have a "subgraph"
# so let's make one

graph = base.create_group(pf.graph, [0, 1])
graph.nodes["subgraph"].__getstate__()

labels ['ElementInput__num', 'SpaceGroupInput__max_structures']
inp:  ElementInput num
GraphEdge(source='va_i_subgraph__ElementInput__num', target='ElementInput', sourceHandle='x', targetHandle='num')
inp:  SpaceGroupInput max_structures
GraphEdge(source='va_i_subgraph__SpaceGroupInput__max_structures', target='SpaceGroupInput', sourceHandle='x', targetHandle='max_structures')
labels ['SpaceGroupInput__dataclass']
node:  ElementInput GraphNode(id='ElementInput', import_path='pyiron_nodes.atomistic.assyst.structures.ElementInput', label='ElementInput', parent_id='subgraph', level=1, node=<pyiron_workflow.simple_workflow.Node object at 0x33b810770>, graph=None, node_type='node', widget_type='customNode', expanded=False)
node:  SpaceGroupInput GraphNode(id='SpaceGroupInput', import_path='pyiron_nodes.atomistic.assyst.structures.SpaceGroupInput', label='SpaceGroupInput', parent_id='subgraph', level=1, node=<pyiron_workflow.simple_workflow.Node object at 0x33af50980>, graph=None, node_type=

{'id': 'subgraph',
 'import_path': None,
 'label': 'subgraph',
 'parent_id': None,
 'level': 0,
 'graph': {'label': 'subgraph',
  'nodes': {'ElementInput': {'id': 'ElementInput',
    'import_path': 'pyiron_nodes.atomistic.assyst.structures.ElementInput',
    'label': 'ElementInput',
    'parent_id': None,
    'level': 0,
    'graph': None,
    'node_type': 'node',
    'widget_type': 'customNode',
    'expanded': False,
    'node': {'label': 'ElementInput',
     'function': 'pyiron_nodes.atomistic.assyst.structures.ElementInput',
     'inputs': {'num': 2}}},
   'SpaceGroupInput': {'id': 'SpaceGroupInput',
    'import_path': 'pyiron_nodes.atomistic.assyst.structures.SpaceGroupInput',
    'label': 'SpaceGroupInput',
    'parent_id': None,
    'level': 0,
    'graph': None,
    'node_type': 'node',
    'widget_type': 'customNode',
    'expanded': False,
    'node': {'label': 'SpaceGroupInput',
     'function': 'pyiron_nodes.atomistic.assyst.structures.SpaceGroupInput',
     'inputs': {'max

In [73]:
# base.graph_to_node(pf.graph.nodes["subgraph"].graph)
base.graph_to_node(graph.nodes["subgraph"].graph)

In [74]:
base._load_graph("executor3", workflow_dir=json_file_location)

Graph(id=None, label='executor3', root_node=None, nodes=Nodes({'Bulk': GraphNode(id='Bulk', import_path='pyiron_nodes.atomistic.structure.build.Bulk', label='Bulk', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33aa0bbc0>, graph=None, node_type='node', widget_type='customNode', expanded=False), 'Volume': GraphNode(id='Volume', import_path='pyiron_nodes.atomistic.structure.calc.Volume', label='Volume', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33aa87ce0>, graph=None, node_type='node', widget_type='customNode', expanded=False), 'ThreadPoolExecutor': GraphNode(id='ThreadPoolExecutor', import_path='pyiron_nodes.executors.ThreadPoolExecutor', label='ThreadPoolExecutor', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33a94b260>, graph=None, node_type='node', widget_type='customNode', expanded=False), 'IterNode': GraphNode(id='IterNode', import_path='pyiron_nodes.executors.IterNode', label

In [75]:
state = pf.graph.nodes["Bulk"].node.__getstate__()
print(state)
pwf.Node().__setstate__(state)

{'label': 'Bulk', 'function': 'pyiron_nodes.atomistic.structure.build.Bulk', 'inputs': {'name': 'Al'}}


In [76]:
dumps = pickle.dumps(pf.graph.nodes)

In [77]:
pickle.loads(dumps)

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,ElementInput,pyiron_nodes.atomistic.assyst.structures.Eleme...,ElementInput,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
1,SpaceGroupInput,pyiron_nodes.atomistic.assyst.structures.Space...,SpaceGroupInput,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
2,SpaceGroupSampling,pyiron_nodes.atomistic.assyst.structures.Space...,SpaceGroupSampling,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
3,ApplyEngine,pyiron_nodes.atomistic.calculator.generic.Appl...,ApplyEngine,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
4,Static,pyiron_nodes.atomistic.calculator.ase.Static,Static,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
5,M3GNet,pyiron_nodes.atomistic.engine.ase.M3GNet,M3GNet,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
6,Bulk,pyiron_nodes.atomistic.structure.build.Bulk,Bulk,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
7,OutputSEFS,pyiron_workflow.api.serial.func_dataclass,OutputSEFS,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
8,Plot,pyiron_nodes.plotting.Plot,Plot,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
9,Histogram,pyiron_nodes.plotting.Histogram,Histogram,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False


In [78]:
pf.graph.nodes

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,ElementInput,pyiron_nodes.atomistic.assyst.structures.Eleme...,ElementInput,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
1,SpaceGroupInput,pyiron_nodes.atomistic.assyst.structures.Space...,SpaceGroupInput,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
2,SpaceGroupSampling,pyiron_nodes.atomistic.assyst.structures.Space...,SpaceGroupSampling,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
3,ApplyEngine,pyiron_nodes.atomistic.calculator.generic.Appl...,ApplyEngine,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
4,Static,pyiron_nodes.atomistic.calculator.ase.Static,Static,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
5,M3GNet,pyiron_nodes.atomistic.engine.ase.M3GNet,M3GNet,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
6,Bulk,pyiron_nodes.atomistic.structure.build.Bulk,Bulk,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
7,OutputSEFS,pyiron_workflow.api.serial.func_dataclass,OutputSEFS,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
8,Plot,pyiron_nodes.plotting.Plot,Plot,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
9,Histogram,pyiron_nodes.plotting.Histogram,Histogram,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False


In [79]:
pf.graph.nodes["Bulk"]

GraphNode(id='Bulk', import_path='pyiron_nodes.atomistic.structure.build.Bulk', label='Bulk', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33b811010>, graph=None, node_type='node', widget_type='customNode', expanded=False)

In [80]:
# node = pf.graph.nodes["subgraph"]
node = graph.nodes["subgraph"]
node.node_type

'graph'

In [81]:
# base.run_macro_node(node.node)
# I've already managed to deprecate this from the codebase
# So let's purge it from the notebooks too
# In any case, you can see from the print above and below that this is a "graph" not a "macro_node"...

print(type(node.node), node.node.node_type)
node.node.run()

<class 'pyiron_workflow.simple_workflow.Node'> graph


SpaceGroupInput(spacegroups=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,

In [82]:
# subgraph = base.get_graph_from_macro_node(node.node)

# In 81, node is `graph.nodes["subgraph"]`
# This is already a "graph"-type node, not a macro node
# So it already has a graph
subgraph = node.graph

subgraph.nodes

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,ElementInput,pyiron_nodes.atomistic.assyst.structures.Eleme...,ElementInput,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
1,SpaceGroupInput,pyiron_nodes.atomistic.assyst.structures.Space...,SpaceGroupInput,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False


I need a way to describe/include virtual nodes both in the code as in the graph representation. Macros do not help since they are given explicitly as a file.
- store original graph + grouping commands
    - makes grouping/ungrouping very easy
    - natural extension of expand/collaps
    - we would then have nodes, edges + groups
    - update would then not only handle expand but also grouping
    - storage etc. would work as before (ideally no extra implementation)
    - issue: how to translate this into code (add virtual nodes as macro to code)
- add macro definitions to code
    - assumption: we do not want to describe all groupings performed by the user, i.e., virtual nodes make sense and are needed
    - pros: human readable, easy to put them later if needed to file
    - question: how does it work together with with run_macro function etc. 

In [83]:
print(base.graph_to_code(pf.graph))


def assyst(ElementInput__num: int = 2, Bulk__name: str = "Al", SpaceGroupInput__max_structures: int = 100, Plot__title: str = "", Plot__symbol: str = "o", Plot__legend_label: str = ""):

    from pyiron_workflow import Workflow
    from pyiron_nodes.atomistic.assyst.structures import ElementInput
    from pyiron_nodes.atomistic.engine.ase import M3GNet
    from pyiron_nodes.atomistic.structure.build import Bulk
    from pyiron_nodes.atomistic.assyst.structures import SpaceGroupInput
    from pyiron_nodes.atomistic.calculator.ase import Static
    from pyiron_nodes.atomistic.assyst.structures import SpaceGroupSampling
    from pyiron_nodes.atomistic.calculator.generic import ApplyEngine
    from pyiron_workflow.api.serial import func_dataclass
    from pyiron_nodes.plotting import Plot
    from pyiron_nodes.plotting import Histogram

    wf = Workflow('assyst')
    wf.ElementInput = ElementInput(num=ElementInput__num)
    wf.M3GNet = M3GNet()
    wf.Bulk = Bulk(name=Bulk__name)
    wf.

In [84]:
node.node_type, node.label

('graph', 'subgraph')

In [85]:
# node._func()

# node has not been redefined since 
#
# # node = pf.graph.nodes["subgraph"]
# node = graph.nodes["subgraph"]
# 
# So it still should not have a func
# Anyhow, other nodes certainly do, but I don't think it's critical that
# we look at this private variable

In [86]:
# base.run_macro_node(node)
node

GraphNode(id='subgraph', import_path=None, label='subgraph', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33aa86180>, graph=Graph(id=None, label='subgraph', root_node=None, nodes=NestedDict({'ElementInput': GraphNode(id='ElementInput', import_path='pyiron_nodes.atomistic.assyst.structures.ElementInput', label='ElementInput', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33b810770>, graph=None, node_type='node', widget_type='customNode', expanded=False), 'SpaceGroupInput': GraphNode(id='SpaceGroupInput', import_path='pyiron_nodes.atomistic.assyst.structures.SpaceGroupInput', label='SpaceGroupInput', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33af50980>, graph=None, node_type='node', widget_type='customNode', expanded=False)}), edges=[GraphEdge(source='ElementInput', target='SpaceGroupInput', sourceHandle='dataclass', targetHandle='element1')], graph={}), node_type='graph', widget_ty

In [87]:
# pf.graph.nodes["IterNode"].node()

select_graph_by_name(pf, "executor")  # Select one of the workflows with an "IterNode"
# It turns out we later need both "Bulk" and "Volume" nodes as well
# "Volume" rules out "executor", so use "executor3"
# Nevermind, "executor3" hits:
# AttributeError: No attribute named Bulk__a

# Invoking `node()` give a no-data error, let's pull
# pf.graph.nodes["IterNode"].node.pull()  # but not like this, that's also a no-data error
base.pull_node(pf.graph, "IterNode")  # Here we go


Running node fcc_metals
Running node ThreadPoolExecutor
Running node IterNode
Restoring node outputs  3f97f129d7455d5bfb7140e81d7dba9da0702921a58096bf334f0081aca8be03 IterNode False
No stored data found for node:  IterNode
copy node (port):  Bulk aed2e0d9ea224e0fb947e9e2b1eda549d2d17da8fc357e6f48e06aa8d5295680
graph_node Bulk
serialization needed


,name,result
0,Cu,"[Atom('Cu', [0.0, 0.0, 0.0], index=0)]"
1,Ag,"[Atom('Ag', [0.0, 0.0, 0.0], index=0)]"
2,Au,"[Atom('Au', [0.0, 0.0, 0.0], index=0)]"
3,Pt,"[Atom('Pt', [0.0, 0.0, 0.0], index=0)]"
4,Ni,"[Atom('Ni', [0.0, 0.0, 0.0], magmom=0.6, index..."
5,Pd,"[Atom('Pd', [0.0, 0.0, 0.0], index=0)]"
6,Rh,"[Atom('Rh', [0.0, 0.0, 0.0], index=0)]"
7,Ir,"[Atom('Ir', [0.0, 0.0, 0.0], index=0)]"


In [ ]:
nodes.atomistic.structure.build.Bulk()._func(name='Al')

In [ ]:
bulk = nodes.atomistic.structure.build.Bulk()
bulk.inputs["name"] = "Al"
bulk.inputs

In [ ]:
nodes.atomistic.structure.build.Bulk()('Al', a=2)

In [ ]:
list(set(['Volume', 'Bulk', 'Volume']))

In [92]:
# We need a workflow with both "Bulk" and "Volume"
# Only "executor3" fits the bill among the loaded flows
# It didn't work to load it initially, but we can switch now
select_graph_by_name(pf, "executor3") 

selected_nodes = ["Bulk", "Volume"]
base.create_group(pf.graph, selected_nodes).nodes["subgraph"].node._workflow # .run()  # graph.nodes["Bulk"].node.run()

labels ['Bulk__name', 'Bulk__a', 'Bulk__cubic']
inp:  Bulk name
GraphEdge(source='va_i_subgraph__Bulk__name', target='Bulk', sourceHandle='x', targetHandle='name')
inp:  Bulk a
GraphEdge(source='va_i_subgraph__Bulk__a', target='Bulk', sourceHandle='x', targetHandle='a')
inp:  Bulk cubic
GraphEdge(source='va_i_subgraph__Bulk__cubic', target='Bulk', sourceHandle='x', targetHandle='cubic')
labels ['Volume__volume']
node:  Bulk GraphNode(id='Bulk', import_path='pyiron_nodes.atomistic.structure.build.Bulk', label='Bulk', parent_id='subgraph', level=1, node=<pyiron_workflow.simple_workflow.Node object at 0x33aa9a5d0>, graph=None, node_type='node', widget_type='customNode', expanded=False)
node:  Volume GraphNode(id='Volume', import_path='pyiron_nodes.atomistic.structure.calc.Volume', label='Volume', parent_id='subgraph', level=1, node=<pyiron_workflow.simple_workflow.Node object at 0x33aa98170>, graph=None, node_type='node', widget_type='customNode', expanded=False)
node:  ThreadPoolExecutor

In [93]:
f = base.create_group(pf.graph, selected_nodes).nodes["subgraph"].graph  #.graph.nodes #.nodes["Bulk"].node.inputs
print(base.graph_to_code(f))

labels ['Bulk__name', 'Bulk__a', 'Bulk__cubic']
inp:  Bulk name
GraphEdge(source='va_i_subgraph__Bulk__name', target='Bulk', sourceHandle='x', targetHandle='name')
inp:  Bulk a
GraphEdge(source='va_i_subgraph__Bulk__a', target='Bulk', sourceHandle='x', targetHandle='a')
inp:  Bulk cubic
GraphEdge(source='va_i_subgraph__Bulk__cubic', target='Bulk', sourceHandle='x', targetHandle='cubic')
labels ['Volume__volume']
node:  Bulk GraphNode(id='Bulk', import_path='pyiron_nodes.atomistic.structure.build.Bulk', label='Bulk', parent_id='subgraph', level=1, node=<pyiron_workflow.simple_workflow.Node object at 0x33aa9a5d0>, graph=None, node_type='node', widget_type='customNode', expanded=False)
node:  Volume GraphNode(id='Volume', import_path='pyiron_nodes.atomistic.structure.calc.Volume', label='Volume', parent_id='subgraph', level=1, node=<pyiron_workflow.simple_workflow.Node object at 0x33aa98170>, graph=None, node_type='node', widget_type='customNode', expanded=False)
node:  ThreadPoolExecutor

In [94]:
f.nodes

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,Bulk,pyiron_nodes.atomistic.structure.build.Bulk,Bulk,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
1,Volume,pyiron_nodes.atomistic.structure.calc.Volume,Volume,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False


In [95]:
base.create_group(pf.graph, selected_nodes)#.nodes["subgraph"].nodes

labels ['Bulk__name', 'Bulk__a', 'Bulk__cubic']
inp:  Bulk name
GraphEdge(source='va_i_subgraph__Bulk__name', target='Bulk', sourceHandle='x', targetHandle='name')
inp:  Bulk a
GraphEdge(source='va_i_subgraph__Bulk__a', target='Bulk', sourceHandle='x', targetHandle='a')
inp:  Bulk cubic
GraphEdge(source='va_i_subgraph__Bulk__cubic', target='Bulk', sourceHandle='x', targetHandle='cubic')
labels ['Volume__volume']
node:  Bulk GraphNode(id='Bulk', import_path='pyiron_nodes.atomistic.structure.build.Bulk', label='Bulk', parent_id='subgraph', level=1, node=<pyiron_workflow.simple_workflow.Node object at 0x33aa9a5d0>, graph=None, node_type='node', widget_type='customNode', expanded=False)
node:  Volume GraphNode(id='Volume', import_path='pyiron_nodes.atomistic.structure.calc.Volume', label='Volume', parent_id='subgraph', level=1, node=<pyiron_workflow.simple_workflow.Node object at 0x33aa98170>, graph=None, node_type='node', widget_type='customNode', expanded=False)
node:  ThreadPoolExecutor

Graph(id=None, label='executor3', root_node=None, nodes=Nodes({'subgraph': GraphNode(id='subgraph', import_path=None, label='subgraph', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33a415b20>, graph=Graph(id=None, label='subgraph', root_node=None, nodes=NestedDict({'Bulk': GraphNode(id='Bulk', import_path='pyiron_nodes.atomistic.structure.build.Bulk', label='Bulk', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33aa9a5d0>, graph=None, node_type='node', widget_type='customNode', expanded=False), 'Volume': GraphNode(id='Volume', import_path='pyiron_nodes.atomistic.structure.calc.Volume', label='Volume', parent_id=None, level=0, node=<pyiron_workflow.simple_workflow.Node object at 0x33aa98170>, graph=None, node_type='node', widget_type='customNode', expanded=False)}), edges=[GraphEdge(source='Bulk', target='Volume', sourceHandle='structure', targetHandle='structure')], graph={}), node_type='graph', widget_type='customNode

In [96]:
# print(get_code_from_graph1(pf.graph))
# NameError: name 'get_code_from_graph1' is not defined

# the one from the library should be ok instead
print(base.get_code_from_graph(pf.graph))


def executor3(Bulk__name: str = "Al", Bulk__a: int = 2, Bulk__cubic: bool = True, IterNode__kwarg_name: str = "Bulk__a", Linspace__x_min: float = 1.8, Linspace__x_max: float = 2.2, Linspace__num_points: int = 7):

    from pyiron_workflow import Workflow
    from pyiron_nodes.atomistic.structure.build import Bulk
    from pyiron_nodes.atomistic.structure.calc import Volume
    from pyiron_nodes.executors import ThreadPoolExecutor
    from pyiron_nodes.executors import IterNode
    from pyiron_nodes.executors import SingleNodeExecutor
    from pyiron_nodes.math import Linspace

    wf = Workflow('executor3')
    wf.Bulk = Bulk(name=Bulk__name, a=Bulk__a, cubic=Bulk__cubic)
    wf.Volume = Volume(structure=wf.Bulk)
    wf.ThreadPoolExecutor = ThreadPoolExecutor()
    wf.IterNode = IterNode(node=wf.Volume, Executor=wf.ThreadPoolExecutor, kwarg_list=wf.Linspace, kwarg_name=IterNode__kwarg_name)
    wf.SingleNodeExecutor = SingleNodeExecutor()
    wf.Linspace = Linspace(x_min=Linspace__x_m

In [97]:
# print(get_code_from_graph(graph, use_node_default=False))

# It looks like the library method `base.get_code_from_graph` was developed here earlier
# We can skip these dead calls without any loss

In [ ]:
@pwf.as_macro_node("out")
def subgraph(Bulk__name: str = "Al"):

    from pyiron_core import nodes
    from pyiron_core.pyiron_workflow import Workflow

    wf = Workflow('subgraph')

    wf.Bulk = nodes.atomistic.structure.build.Bulk(name=Bulk__name)
    wf.Volume = nodes.atomistic.structure.calc.Volume(structure=wf.Bulk)

    return wf.Volume

subgraph()(Bulk__name="Al")

In [ ]:
subgraph().node_type

In [100]:
graph = pf.graph
graph = base.topological_sort(graph)

base.get_non_default_input(graph, flatten=True)

[('Bulk', 'name'),
 ('Bulk', 'a'),
 ('Bulk', 'cubic'),
 ('Linspace', 'x_min'),
 ('Linspace', 'x_max'),
 ('Linspace', 'num_points'),
 ('Volume', 'structure'),
 ('IterNode', 'node'),
 ('IterNode', 'kwarg_name'),
 ('IterNode', 'kwarg_list'),
 ('IterNode', 'Executor')]

In [101]:
# Reload the module
# importlib.reload(to_code)

print(base.get_code_from_graph(f, use_node_default=False))


def subgraph(Bulk__name: str = "Al", Bulk__a: int = 2, Bulk__cubic: bool = True):

    from pyiron_workflow import Workflow
    from pyiron_nodes.atomistic.structure.build import Bulk
    from pyiron_nodes.atomistic.structure.calc import Volume

    wf = Workflow('subgraph')
    wf.Bulk = Bulk(name=Bulk__name, a=Bulk__a, cubic=Bulk__cubic)
    wf.Volume = Volume(structure=wf.Bulk)

    return wf.Volume.outputs.volume



In [102]:
# black.format_str()
# TypeError: format_str() missing 1 required positional argument: 'src_contents'

# Just feed it something
black.format_str("def f(arg:str='')->None:...", mode=black.Mode())

'def f(arg: str = "") -> None: ...\n'

In [103]:
mode = black.Mode()

# print(black.format_str(get_code_from_graph(pf.graph), mode=mode))
#  ValueError: Macro works only for single output node.

# Per https://github.com/JNmpi/pyiron_core/issues/33, this is not "macros"
# This is some part of the auto-parser (also used for grouping subgraphs) only accepting single returns
# True macros can indeed return multiple items
# Let's select a graph where this might work and try again

select_graph_by_name(pf, "linearfit2")
print(black.format_str(base.get_code_from_graph(pf.graph), mode=mode))

def linearfit2(ReadPickledDatasetAsDataframe__file_path: str = "mgca.pckl.tgz"):

    from pyiron_workflow import Workflow
    from pyiron_nodes.atomistic.ml_potentials.fitting.linearfit import (
        ParameterizePotentialConfig,
    )
    from pyiron_nodes.atomistic.ml_potentials.fitting.linearfit import (
        SplitTrainingAndTesting,
    )
    from pyiron_nodes.atomistic.ml_potentials.fitting.linearfit import RunLinearFit
    from pyiron_nodes.atomistic.ml_potentials.fitting.linearfit import (
        ReadPickledDatasetAsDataframe,
    )
    from pyiron_nodes.atomistic.ml_potentials.fitting.linearfit import (
        PlotEnergyFittingCurve,
    )
    from pyiron_nodes.atomistic.ml_potentials.fitting.linearfit import (
        PredictEnergiesAndForces,
    )

    wf = Workflow("linearfit2")
    wf.ParameterizePotentialConfig = ParameterizePotentialConfig()
    wf.SplitTrainingAndTesting = SplitTrainingAndTesting(
        data_df=wf.ReadPickledDatasetAsDataframe
    )
    wf.Run

In [ ]:

# Without this, when I try to reference pyiron_nodes.atomistic.property.calphy.X`, I get
# AttributeError: module 'pyiron_nodes.atomistic.property' has no attribute 'calphy'
# This must just be something with the init files not registering modules nicely over in pyiron_nodes
# My notebook is highlighting `property` as a python keyword, so maybe there is some conflict that 
# confuses things?
# Still, we never need to actually use pn_calphy, just completing the import is enough


wf = pwf.Workflow('calphy2')

element = "Al"
cell_size = 3
solid_fraction = 0

wf.CubicBulkCell = nodes.atomistic.structure.build.CubicBulkCell(element=element, cell_size=cell_size)
wf.Potential = nodes.atomistic.engine.lammps.Potential(structure=wf.CubicBulkCell)

wf.Tolerance = nodes.atomistic.property.calphy.Tolerance(solid_fraction=solid_fraction)
wf.InputClass = nodes.atomistic.property.calphy.InputClass()
wf.InputClass_1 = nodes.atomistic.property.calphy.InputClass(tolerance=wf.Tolerance)
wf.SolidFreeEnergyWithTemperature = nodes.atomistic.property.calphy.SolidFreeEnergyWithTemperature(structure=wf.CubicBulkCell, potential=wf.Potential, inp=wf.InputClass_1, store=False)
wf.LiquidFreeEnergyWithTemperature = nodes.atomistic.property.calphy.LiquidFreeEnergyWithTemperature(potential=wf.Potential, structure=wf.CubicBulkCell, inp=wf.InputClass, store=False)
wf.CalculatePhaseTransformationTemperature = nodes.atomistic.property.calphy.CalculatePhaseTransformationTemperature(t1=wf.SolidFreeEnergyWithTemperature.outputs.t,
                                                                                                                            f1=wf.SolidFreeEnergyWithTemperature.outputs.f, 
                                                                                                                            t2=wf.LiquidFreeEnergyWithTemperature.outputs.t, 
                                                                                                                            f2=wf.LiquidFreeEnergyWithTemperature.outputs.f) 

wf.CalculatePhaseTransformationTemperature.pull()

In [ ]:
# # graph = full_graph = get_full_graph_from_wf(wf)
# graph = _load_graph("calphy2.json")
# Loading still doesn't work: https://github.com/JNmpi/pyiron_core/issues/52
# Let's use the commented-out running of the workflow we made here
graph = base.get_full_graph_from_wf(wf)


base.pull_node(base.get_updated_graph(graph), "LiquidFreeEnergyWithTemperature")

critical feature:
- executorlib must be able to run single node or (sub)-workflow
- executorlib is a task manager (not a workflow executor)
- library stack:
    - workflow_creator
    - workflow_executor
    - executor_lib
- application:
    - exec.submit(node)  

In [ ]:
@pwf.as_macro_node("transition_temperature")
def calphy2(element: str, cell_size: int = 1, solid_fraction: float = 0.7):

    from pyiron_core import nodes
    from pyiron_core.pyiron_workflow import Workflow

    wf = Workflow('calphy2')

    wf.CubicBulkCell = nodes.atomistic.structure.build.CubicBulkCell(element=element, cell_size=cell_size)
    wf.Tolerance = nodes.atomistic.property.calphy.Tolerance(solid_fraction=solid_fraction)
    wf.InputClass = nodes.atomistic.property.calphy.InputClass()
    wf.Potential = nodes.atomistic.engine.lammps.Potential(structure=wf.CubicBulkCell)
    wf.InputClass_1 = nodes.atomistic.property.calphy.InputClass(tolerance=wf.Tolerance)
    wf.LiquidFreeEnergyWithTemperature = nodes.atomistic.property.calphy.LiquidFreeEnergyWithTemperature(potential=wf.Potential, structure=wf.CubicBulkCell, inp=wf.InputClass)
    wf.SolidFreeEnergyWithTemperature = nodes.atomistic.property.calphy.SolidFreeEnergyWithTemperature(structure=wf.CubicBulkCell, potential=wf.Potential, inp=wf.InputClass_1)
    wf.CalculatePhaseTransformationTemperature = nodes.atomistic.property.calphy.CalculatePhaseTransformationTemperature(t1=wf.SolidFreeEnergyWithTemperature.outputs.t, f1=wf.SolidFreeEnergyWithTemperature.outputs.f, t2=wf.LiquidFreeEnergyWithTemperature.outputs.t, f2=wf.LiquidFreeEnergyWithTemperature.outputs.f)

    return wf.CalculatePhaseTransformationTemperature# .outputs.transition_temp #, wf.CalculatePhaseTransformationTemperature.outputs.plot

    

It appears that macro of a macro does not work
- check!!

It does now! Calphy runs are costly, so we demo this separately now

Aside on macros within macros:

In [107]:
@pwf.as_function_node
def AddOne(x):
    y = x + 1
    return y
    
@pwf.as_macro_node(["y"])
def AddTwo(x):
    wf = pwf.Workflow("add_two_subgraph")
    wf.n1 = AddOne(x)
    wf.n2 = AddOne(wf.n1)
    return wf.n2

@pwf.as_macro_node(["y"])
def AddThree(x):
    wf = pwf.Workflow("add_three_subgraph")
    wf.n1 = AddOne(x)
    wf.n2 = AddTwo(x)
    return wf.n2

wf = pwf.Workflow("nested_macro_demo")
wf.nested = AddThree(0)
wf.run()

2

Note that nesting graph groups is currently still problematic: https://github.com/JNmpi/pyiron_core/issues/50

End aside on macros within macros

In [ ]:
@pwf.as_macro_node("transition_temperature")
def calphy2(element: str, cell_size: int = 1, solid_fraction: float = 0.7):

    from pyiron_core import nodes
    from pyiron_core.pyiron_workflow import Workflow

    wf = Workflow('calphy2')

    # wf.CubicBulkCell = pyiron_nodes.atomistic.structure.build.Bulk("Al") # CubicBulkCell(element=element, cell_size=cell_size) 
    wf.CubicBulkCell = nodes.atomistic.structure.build.CubicBulkCell(element=element, cell_size=cell_size)

    return wf.CubicBulkCell

In [ ]:
graph = base.Graph()
graph += nodes.atomistic.structure.build.CubicBulkCell(element="Al")
graph.nodes

In [ ]:
n = calphy2(element="Al")
graph = base.get_graph_from_macro_node(n)
# base.run_macro_node(n)
# base.GuiGraph(base.get_graph_from_macro_node(n))
# base.get_graph_from_macro_node(n).nodes["CubicBulkCell"].node.run()
graph.nodes

In [ ]:
@pwf.as_macro_node("structure")
def CubicBulkCell(
    element: str, cell_size: int = 1, vacancy_index: Optional[int] = None
):

    from pyiron_core.pyiron_nodes.atomistic.structure.build import Bulk
    from pyiron_core.pyiron_nodes.atomistic.structure.transform import (
        CreateVacancy,
        Repeat,
    )
    from pyiron_core.pyiron_workflow import Workflow

    wf = Workflow('macro')

    wf.bulk = Bulk(name=element, cubic=True)
    wf.repeat = Repeat(structure=wf.bulk, repeat_scalar=cell_size)

    wf.vacancy = CreateVacancy(structure=wf.repeat, index=vacancy_index)
    return wf.vacancy

In [ ]:
CubicBulkCell("Al").run()

In [ ]:
nodes.atomistic.structure.build.CubicBulkCell(element="Al")

In [ ]:
%%time
with executorlib.SingleNodeExecutor(max_workers=4, block_allocation=True) as exe:
    future_lst = [exe.submit(sum, [i, i]) for i in range(2, 5)]
    print([f.result() for f in future_lst])

In [115]:
PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True




In [116]:
%%time
with executorlib.SingleNodeExecutor(max_workers=6, block_allocation=False) as exe:
    future_lst = [exe.submit(is_prime, n) for n in PRIMES]
    print([f.result() for f in future_lst])

[True, True, True, True, True, False]
CPU times: user 44.5 ms, sys: 33.8 ms, total: 78.3 ms
Wall time: 311 ms


In [ ]:
%%time
with executorlib.SingleNodeExecutor(max_workers=1, block_allocation=True) as exe:
    future_lst = [exe.submit(nodes.atomistic.structure.build.Bulk()._func, n) for n in ["Al", "Fe", "Ni", "Zn"]]
    print([f.result() for f in future_lst])

In [ ]:
Al = nodes.atomistic.structure.build.Bulk()._func('Al')
Al.species[0].name

In [ ]:
%%time
for n in ["Al", "Fe"]:
    print(nodes.atomistic.structure.build.Bulk(n).run())

In [ ]:
elements = ['Al', 'Fe']


# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    # Start the load operations and mark each future with its URL
    futures = {executor.submit(nodes.atomistic.structure.build.Bulk()._func, el, **{'cubic': True}): el for el in elements}
    for future in concurrent.futures.as_completed(futures):
        out = futures[future]
        print("out: ", out)

        data = future.result()
        print(data.__repr__())

In [ ]:
@pwf.as_function_node
def IterNode(node: pwf.Node, kwarg_name: str, kwarg_list: list, max_workers: int=1, executor=None):
    # from concurrent.futures import ThreadPoolExecutor as Executor
    from executorlib import SingleNodeExecutor as Executor
    from concurrent.futures import as_completed

    out_dict = dict()
    with Executor(max_workers=max_workers) as executor:
        # Start the load operations and mark each future with its index
        futures = {executor.submit(node._func, **{kwarg_name: el}): el for el in kwarg_list}
        for future in as_completed(futures):
            out = futures[future]    
            data = future.result()
            out_dict[out] = data.species[0].name
            
            # print("out: ", out)
            # print(data.__repr__())
            
    return out_dict
    

In [ ]:
%%time
elements = ['Al', 'Fe']

IterNode(nodes.atomistic.structure.build.Bulk(), "name", elements, max_workers=2).run()

In [ ]:
# Define the matrix mat and the index array ind1
mat = np.array([[0, 1, 2],
                [3, 4, 5],
                [6, 7, 8]])

ind1 = np.array([1, 0, 2])

# Compute v using advanced indexing
v = mat[ind1, np.arange(len(ind1))]

v  # Output: [3, 1, 8]

In [ ]:
n = nodes.atomistic.structure.build.Bulk("Al")

In [ ]:
kwargs = n.kwargs
kwargs.update({"a": 2.1})
kwargs

In [ ]:
# CLEANUP

lpg.cleanup()

cwd = pathlib.Path("../liam")

with contextlib.suppress(FileNotFoundError):
    (cwd / "NiAlH_jea.eam.alloy").unlink()

with contextlib.suppress(FileNotFoundError):
    (cwd / "pyiron.log").unlink()

for path in cwd.glob("*.dat"):
    with contextlib.suppress(FileNotFoundError):
        path.unlink()

for path in cwd.glob("*.dat-*"):
    if path.is_dir():
        with contextlib.suppress(FileNotFoundError):
            shutil.rmtree(path)

path = (cwd / ".storage")
if path.is_dir():
    with contextlib.suppress(FileNotFoundError):
        shutil.rmtree(path)